In [119]:
import pandas as pd
import plotly as pt
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [67]:
confirmed_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')

country_coordinates = {}

for index,row in confirmed_cases.iterrows():
    if row['Country/Region'] in country_coordinates:
        pass
    else:
        country_coordinates[row['Country/Region']] = (row.Lat, row.Long)

In [68]:
confirmed_cases_by_country = confirmed_cases.groupby("Country/Region").sum(numeric_only=True)\
                                            .drop(columns=['Lat', 'Long'])

In [69]:
def date_converter(date_str):
    month_map = {'1': "January", '2': "February", '3': "March", '4': "April", '5': "May", '6': "June", 
                 '7': "July", '8': "August", '9': "September", '10': "October", '11': "November", '12': "December"}

    split_date = date_str.split('/')

    return month_map[split_date[0]] + split_date[-1]

In [70]:
month_list = []
for i in confirmed_cases_by_country.columns:
    month_list.append(date_converter(i))

confirmed_cases_by_country.columns = month_list

In [95]:
monthly_data_df = pd.DataFrame()

for month in pd.unique(confirmed_cases_by_country.columns):
    x = confirmed_cases_by_country[month]
    monthly_data_df[month] = x.max(axis=1)

monthly_data_df.loc['US']

January20              8
February20            25
March20           192079
April20          1081150
May20            1791547
June20           2648797
July20           4548497
August20         6046489
September20      7246200
October20        9176352
November20      13667915
December20      20219878
January21       26357107
February21      28764298
March21         30579485
April21         32468055
May21           33386447
June21          33782325
July21          35107404
August21        39395088
September21     43539596
October21       46053361
November21      48606339
December21      54907717
January22       75244152
February22      79181996
March22         80221585
April22         81466441
May22           84327544
June22          87675415
July22          91345425
August22        94528255
September22     96361580
October22       97491766
November22      98813321
December22     100765333
January23      102362870
February23     103443455
March23        103802702
Name: US, dtype: int64

In [134]:
globe = folium.Map(location=[54.5, 15.25], zoom_start=4)

final_df = monthly_data_df.max(axis=1).to_frame(name='max_cases')
final_df['ranked'] = (np.log(final_df['max_cases']).round(2))/2 

for country in country_coordinates:
    folium.CircleMarker(
        location=country_coordinates[country],
        radius=final_df.loc[country, 'ranked'],
        fill=True,
        popup=folium.Popup("inline explicit Popup"),
    ).add_to(globe)

display(globe)

9.23